# Data Cleaning / Wrangling

In [31]:
# Load libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as datetime

In [2]:
# Read in data
towerseasons = pd.read_csv('tower_seasons_2019-11-21.csv', parse_dates=['max_canopy_date'])
towerweekly = pd.read_csv('tower_weekly_2019-11-22.csv', parse_dates=['date'])
towers = pd.read_csv('towers_2019-11-21.csv', parse_dates = ['installed_at'])

In [37]:
# Drop NAs
towerseasons_clean = towerseasons.dropna()
towerweekly_clean = towerweekly.dropna().copy()
towers_clean = towers.dropna()

In [5]:
# View data
towerseasons_clean.head(5)

,tower_id,year,actual_max_canopy,max_canopy_date
2,3512,2019,0.458826,2019-07-08
13,825,2019,0.659543,2019-07-01
26,2048,2019,0.660164,2019-06-17
31,3994,2019,0.700000,2019-05-27
32,5442,2019,0.900000,2019-10-21


In [7]:
towerweekly_clean.head(5)

,tower_id,date,irrigation_mm,actual_precip_mm,pri
0,10,2019-01-07,0.0,14.204270,1.183220
1,10,2019-01-14,0.0,85.758867,1.158736
2,10,2019-01-21,0.0,0.000000,1.063871
3,10,2019-01-28,0.0,43.022252,1.088352
4,10,2019-02-04,0.0,19.813417,1.166881


In [8]:
towers_clean.head(5)

,tower_id,crop,metacrop,betweenrowft,slope,aspect,region,subregion,installed_at,young_plant,trellis_type
0,4068,grape,vine,5.0,4,85,NCoast,napa,2019-04-03 18:42:35,True,vsp
1,3490,grape,vine,8.0,0,297,NCoast,napa,2018-04-19 22:57:42,False,vsp
2,1297,grape,vine,10.0,1,252,NCoast,napa,2016-05-20 19:16:29,False,lyre
3,61,grape,vine,7.0,0,0,NCoast,napa,2014-04-30 04:00:00,False,vsp
4,20,grape,vine,8.0,0,0,NCoast,napa,2014-04-21 04:00:00,False,lyre


In [9]:
# Convert year to datetime
towerweekly_clean['year'] = pd.DatetimeIndex(towerweekly_clean['date']).year
towerweekly_clean.head()

,tower_id,date,irrigation_mm,actual_precip_mm,pri,year
0,10,2019-01-07,0.0,14.204270,1.183220,2019
1,10,2019-01-14,0.0,85.758867,1.158736,2019
2,10,2019-01-21,0.0,0.000000,1.063871,2019
3,10,2019-01-28,0.0,43.022252,1.088352,2019
4,10,2019-02-04,0.0,19.813417,1.166881,2019


## Cleaning

### For about half of the towers, one of the weekly pri values in this table corresponds to the actual_max_canopy in the tower_seasons table. For the other half of the towers, we had to manually overwrite the max canopy value and date using an admin tool. There is a defect in the admin tool, such that it calculated the weekly PRI differently that the weekly PRI that is in our database, so any overwritten value is slightly different that what can be found by querying our database. There are matches between the tower_weekly data and tower_seasons data for 600 sites, so it should be sufficient. Please discard the towers where one of the weekly values in tower_weekly does not match the actual_max_canopy value in tower_seasons. That is, please do not include them in your analysis. The clean up step here is to throw the data for these tower_ids away.

In [15]:
## Identifying PRI and max canopy values that don't match and filtering them out. Major cleaning step

In [16]:
# Step 1 - Narrow down to columns of interest
towerweekly_group_pri_clean= towerweekly_clean[['year', 'tower_id', 'pri']]
# towerweekly_group_pri_clean.head()

In [17]:
# Step 2 - Merge previous dataframe with tower_seasons 
tower_pri_comb = towerseasons_clean.merge(towerweekly_group_pri_clean, on =['tower_id', 'year'])
# tower_pri_comb.head()

In [18]:
# Step 3 - Create a new column 'different' which calculates difference between max canopy and PRI
# Remember we want values that are NOT different aka the difference should equal 0 
tower_pri_comb['different'] = tower_pri_comb.actual_max_canopy - tower_pri_comb.pri
# tower_pri_comb.head()

In [19]:
# Step 4 - How many cases did NOT equal 0?
len(tower_pri_comb[tower_pri_comb['different'] != 0 ])

31345

In [20]:
# Step 5 - More importantly, how many cases did equal 0? Almost 600, which is about what we expected.
len(tower_pri_comb[tower_pri_comb['different'] == 0 ])

594

In [21]:
# Step 6 - This is your new dataframe 
trimmed_tower_weekly = tower_pri_comb[tower_pri_comb['different'] == 0 ].copy()
trimmed_tower_weekly.head()

,tower_id,year,actual_max_canopy,max_canopy_date,pri,different
26,3512,2019,0.458826,2019-07-08,0.458826,0.0
89,2048,2019,0.660164,2019-06-17,0.660164,0.0
187,832,2019,0.515444,2019-07-08,0.515444,0.0
205,4230,2019,0.580303,2019-07-08,0.580303,0.0
319,2305,2019,0.701330,2019-06-10,0.701330,0.0


In [24]:
# How large is the dataframe we have post cleaning?
len(trimmed_tower_weekly)

594

In [25]:
# Do the same exact thing but add the other variables back in.
# Redundant but keeping it in case someone wants the freedom to complete everything in one step. 
# towerweekly_group_pri_clean_complete = towerweekly_clean[['year', 'tower_id', 'pri', 'irrigation_mm', 'actual_precip_mm', 'date']]
# tower_pri_comb_complete = towerseasons_clean.merge(towerweekly_group_pri_clean_complete, on =['tower_id', 'year'])
# tower_pri_comb_complete['different'] = tower_pri_comb_complete.actual_max_canopy - tower_pri_comb_complete.pri
# len(tower_pri_comb_complete[tower_pri_comb_complete['different'] != 0 ])
# len(tower_pri_comb_complete[tower_pri_comb_complete['different'] == 0 ])
# trimmed_tower_weekly_complete = tower_pri_comb_complete[tower_pri_comb_complete['different'] == 0 ].copy()
# trimmed_tower_weekly.head()
# len(trimmed_tower_weekly)

In [26]:
# Groupby statement for relevant stats
towerweekly_group = trimmed_tower_weekly.groupby(['year', 'tower_id']).mean()
towerweekly_group.head()

actual_max_canopy       pri  different
year tower_id                                        
2019 10                 0.500498  0.500498        0.0
     18                 0.768882  0.768882        0.0
     20                 0.813847  0.813847        0.0
     21                 0.559340  0.559340        0.0
     27                 0.693847  0.693847        0.0

In [ ]:
# Lets view the size of the dataframes

In [ ]:
print("Towers dimensions: {}".format(trimmed_tower_weekly_complete.shape))
print("Towers Weekly: {}".format(towerseasons_clean.shape))
print("Towers Seasonal: {}".format(towers_clean.shape))

In [ ]:
# Lets now combine them to make a "final table"

In [27]:
trimmed_tower_weekly_complete.head()

,tower_id,year,actual_max_canopy,max_canopy_date,pri,irrigation_mm,actual_precip_mm,date,different
26,3512,2019,0.458826,2019-07-08,0.458826,0.000000,0.0,2019-07-08,0.0
89,2048,2019,0.660164,2019-06-17,0.660164,2.020265,0.0,2019-06-17,0.0
187,832,2019,0.515444,2019-07-08,0.515444,0.000000,0.0,2019-07-08,0.0
205,4230,2019,0.580303,2019-07-08,0.580303,6.047731,0.0,2019-07-08,0.0
319,2305,2019,0.701330,2019-06-10,0.701330,0.000000,0.0,2019-06-10,0.0


In [28]:
# Merge previous table with towers on tower_id
# Is a fairly small data frame. Something to consider. 
final_table = pd.merge(trimmed_tower_weekly_complete, towers_clean, on = 'tower_id')
print("Towers Seasonal: {}".format(final_table.shape))

Towers Seasonal: (377, 19)


In [30]:
# View final table
final_table.head()

,tower_id,year,actual_max_canopy,max_canopy_date,pri,irrigation_mm,actual_precip_mm,date,different,crop,metacrop,betweenrowft,slope,aspect,region,subregion,installed_at,young_plant,trellis_type
0,3512,2019,0.458826,2019-07-08,0.458826,0.000000,0.0,2019-07-08,0.0,grape,vine,7.0,2,198,NCoast,carneros,2018-06-07 22:24:00,False,quad
1,2048,2019,0.660164,2019-06-17,0.660164,2.020265,0.0,2019-06-17,0.0,grape,vine,8.0,1,206,CCoast,pasorobles,2017-03-09 19:09:41,True,vsp
2,832,2019,0.515444,2019-07-08,0.515444,0.000000,0.0,2019-07-08,0.0,grape,vine,8.0,2,190,NCoast,carneros,2016-03-03 00:25:00,False,vsp
3,4230,2019,0.580303,2019-07-08,0.580303,6.047731,0.0,2019-07-08,0.0,grape,vine,8.0,10,335,NCoast,carneros,2019-04-19 18:47:10,False,vsp
4,2305,2019,0.701330,2019-06-10,0.701330,0.000000,0.0,2019-06-10,0.0,grape,vine,9.0,1,225,CCoast,pasorobles,2017-04-26 22:30:47,True,vsp


In [42]:
# Save the dataframe
final_table.to_csv(r'/Users/ajeustis/Downloads/final_table.csv', index=False)
towerseasons_clean.to_csv(r'/Users/ajeustis/Downloads/towerseasons_clean.csv', index=False)
towerweekly_clean.to_csv(r'/Users/ajeustis/Downloads/towerweekly_clean.csv', index=False)
towers_clean.to_csv(r'/Users/ajeustis/Downloads/towers_clean.csv', index=False)

In [49]:
# Check to make sure they were saved properly

In [48]:
final_table.head()

,tower_id,year,actual_max_canopy,max_canopy_date,pri,irrigation_mm,actual_precip_mm,date,different,crop,metacrop,betweenrowft,slope,aspect,region,subregion,installed_at,young_plant,trellis_type
0,3512,2019,0.458826,2019-07-08,0.458826,0.000000,0.0,2019-07-08,0.0,grape,vine,7.0,2,198,NCoast,carneros,2018-06-07 22:24:00,False,quad
1,2048,2019,0.660164,2019-06-17,0.660164,2.020265,0.0,2019-06-17,0.0,grape,vine,8.0,1,206,CCoast,pasorobles,2017-03-09 19:09:41,True,vsp
2,832,2019,0.515444,2019-07-08,0.515444,0.000000,0.0,2019-07-08,0.0,grape,vine,8.0,2,190,NCoast,carneros,2016-03-03 00:25:00,False,vsp
3,4230,2019,0.580303,2019-07-08,0.580303,6.047731,0.0,2019-07-08,0.0,grape,vine,8.0,10,335,NCoast,carneros,2019-04-19 18:47:10,False,vsp
4,2305,2019,0.701330,2019-06-10,0.701330,0.000000,0.0,2019-06-10,0.0,grape,vine,9.0,1,225,CCoast,pasorobles,2017-04-26 22:30:47,True,vsp


In [46]:
towerseasons_clean.head()

,tower_id,year,actual_max_canopy,max_canopy_date
2,3512,2019,0.458826,2019-07-08
13,825,2019,0.659543,2019-07-01
26,2048,2019,0.660164,2019-06-17
31,3994,2019,0.700000,2019-05-27
32,5442,2019,0.900000,2019-10-21


In [47]:
towerweekly_clean.head()

,tower_id,date,irrigation_mm,actual_precip_mm,pri
0,10,2019-01-07,0.0,14.204270,1.183220
1,10,2019-01-14,0.0,85.758867,1.158736
2,10,2019-01-21,0.0,0.000000,1.063871
3,10,2019-01-28,0.0,43.022252,1.088352
4,10,2019-02-04,0.0,19.813417,1.166881


In [50]:
towers_clean.head()

,tower_id,crop,metacrop,betweenrowft,slope,aspect,region,subregion,installed_at,young_plant,trellis_type
0,4068,grape,vine,5.0,4,85,NCoast,napa,2019-04-03 18:42:35,True,vsp
1,3490,grape,vine,8.0,0,297,NCoast,napa,2018-04-19 22:57:42,False,vsp
2,1297,grape,vine,10.0,1,252,NCoast,napa,2016-05-20 19:16:29,False,lyre
3,61,grape,vine,7.0,0,0,NCoast,napa,2014-04-30 04:00:00,False,vsp
4,20,grape,vine,8.0,0,0,NCoast,napa,2014-04-21 04:00:00,False,lyre
